In [29]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the folders for figures, estimations, and real parameters
figures_folder = os.path.join("figures", "box")
replications_folder = "estimation1/parameters"

# Create the figures folder if it doesn't exist
os.makedirs(figures_folder, exist_ok=True)

# Define the sizes of the series
sizes = [200, 500, 1000]
models = ['M2']

parameter_names = ["omega1", "alpha11", "beta11", "omega2", 
                   "alpha22","beta22", "delta"]

# Define min and max values for each parameter (adjust these as needed)
parameter_ranges = {
    "omega1": (-0.4, 1.2),
    "alpha11": (-0.25, 0.6),
    "beta11": (0.2, 0.8),
    "omega2": (-0.3, 1.6),
    "alpha22": (-0.3, 0.9),
    "beta22": (0, 0.7),  # Adjust these limits if needed
    "delta": (-0.2, 2.1)
}

# Loop through each model
for model in models:
    estimation_folder = f"estimation1/{model}"
    estimation_folder2 = f"replications1/{model}"

    fig, axs = plt.subplots(len(parameter_names), 1, figsize=(15, 12 * len(parameter_names)), sharex=True)
    fig.suptitle(f"{model}", fontsize=30)

    for param_idx, param_name in enumerate(parameter_names):
        # Loop through each size
        for size_idx, size in enumerate(sizes):
            size_folder = os.path.join(estimation_folder2, f"size_{size}")

            # Load parameter data (ensure it is numeric)
            filename = f"modelestim_{model}_{size}.csv"
            filepath = os.path.join(size_folder, filename)
            try:
                parameter_data = np.loadtxt(filepath, delimiter=',')
            except Exception as e:
                print(f"Error loading parameter data from {filepath}: {e}")
                continue

            # Load the estimated parameters
            try:
                df_estimated = pd.read_csv(
                    f"{estimation_folder}/theta_hat_{model}_{size}.csv", header=None)
            except Exception as e:
                print(f"Error loading estimated parameters: {e}")
                continue

            # Load the real parameters
            try:
                df_real = pd.read_csv(
                    f"{replications_folder}/real_parameters_{model}.csv", header=None)
            except Exception as e:
                print(f"Error loading real parameters: {e}")
                continue

            # Create boxplots for each size and estimation method
            bp1 = axs[param_idx].boxplot(df_estimated.iloc[:, param_idx].dropna(), vert=True, patch_artist=True,
                                         positions=[size_idx * 3 + 1], widths=0.8,
                                         boxprops=dict(
                                             facecolor='limegreen', color='green'),
                                         whiskerprops=dict(color='green'),
                                         capprops=dict(color='green'),
                                         medianprops=dict(color='purple'))

            if param_idx < len(parameter_data):  # Check if the index is valid
                bp2 = axs[param_idx].boxplot(parameter_data[param_idx, :], vert=True, patch_artist=True,
                                             positions=[size_idx * 3 + 2], widths=0.8,
                                             boxprops=dict(
                                                 facecolor='lightblue', color='blue'),
                                             whiskerprops=dict(color='blue'),
                                             capprops=dict(color='blue'),
                                             medianprops=dict(color='purple'))

            # Add a line for the real parameter value
            axs[param_idx].axhline(df_real.iloc[param_idx, 0], color='red', linestyle='--',
                                   linewidth=2, label='Vraie Valeur')

            # Set subplot title
            axs[param_idx].set_title(f"{parameter_names[param_idx]}", fontsize=20)

            # Set y-axis limits based on parameter range
            min_val, max_val = parameter_ranges[param_name]
            axs[param_idx].set_ylim(min_val, max_val)

        # Set x-axis ticks and labels for each subplot
        axs[param_idx].set_xticks([1.5, 4.5, 7.5])
        axs[param_idx].set_xticklabels(sizes, fontsize=20)

    # Set the x-axis label only for the bottom subplot
    axs[-1].set_xlabel('Taille de l\'échantillon', fontsize=20)

    # Set the y-axis label for all subplots
    for ax in axs:
        ax.set_ylabel('Valeurs des paramètres', fontsize=20)

    # Create a single legend for the entire figure
    handles = [plt.Line2D([], [], color='lightblue', linestyle='-', linewidth=5, label='MCO'),
               plt.Line2D([], [], color='limegreen', linestyle='-',
                          linewidth=5, label='MV'),
               plt.Line2D([], [], color='red', linestyle='--',
                          linewidth=2, label='Vraie Valeur')]
    fig.legend(handles=handles, loc='upper right', fontsize=25)

    # Fine-tune spacing
    plt.subplots_adjust(hspace=0.3, top=0.95, bottom=0.1, left=0.1, right=0.95)  # Adjust top 

    # Save the plot to the figures folder
    plot_filename = f"combined_boxplot_{model}_all_parameters.png"
    plot_filepath = os.path.join(figures_folder, plot_filename)
    plt.savefig(plot_filepath)
    plt.close(fig)

    print(f"Saved combined box plot to: {plot_filepath}")

Saved combined box plot to: figures\box\combined_boxplot_M2_all_parameters.png
